In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="p1")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-142925
Azure region: southcentralus
Subscription id: f9d5a085-54dc-4215-9ba6-dad5d86e60a0
Resource group: aml-quickstarts-142925


### (Optional) Cancel existing runs

In [11]:
for run in exp.get_runs():
    print(run.id)
    if run.status=="Running":
        run.cancel()

HD_93afb055-83d9-41a7-9c50-e9a7e14cd79d
HD_a78b0e0e-008f-4804-9296-85be82ed1739
7dcaf971-c29d-4f68-b72e-0815db91a92d


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "udacityAzureML"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

print(compute_target.get_status().serialize())

Found existing compute target

Running
{'errors': [], 'creationTime': '2021-04-17T10:57:19.241877+00:00', 'createdBy': {'userObjectId': 'f3c9ec0e-655a-4344-8b94-baf1544d5426', 'userTenantId': '660b3398-b80e-49d2-bc5b-ac1dc93b5254', 'userName': 'ODL_User 142925'}, 'modifiedTime': '2021-04-17T10:59:36.333182+00:00', 'state': 'Running', 'vmSize': 'STANDARD_DS2_V2'}


In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
import os

ps = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1,10,100),
        '--max_iter': choice(50,100,200)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = "./",
            compute_target=compute_target,
            vm_size='STANDARD_D2_V2',
            entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     estimator=est,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [12]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")

RunId: HD_4b351f6b-88c0-46ea-abf3-3a48ed4bf074
Web View: https://ml.azure.com/runs/HD_4b351f6b-88c0-46ea-abf3-3a48ed4bf074?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-142925/workspaces/quick-starts-ws-142925&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-04-17T11:37:19.465848][API][INFO]Experiment created<END>\n""<START>[2021-04-17T11:37:20.345133][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-04-17T11:37:20.678429][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-04-17T11:37:20.8164067Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_4b351f6b-88c0-46ea-abf3-3a48ed4bf074
Web View: https://ml.azure.com/runs/HD_4b351f6b-88c0-46ea-abf3-3a48ed4bf074?wsid=/subscriptions/f9d5a085-54dc-4215-9

In [14]:
import joblib
best_run = hyperdrive_run.get_best_run_by_primary_metric()

print("Best run metrics :",best_run.get_metrics())
print("Best run details :",best_run.get_details())
print("Best run file names :",best_run.get_file_names())

model = best_run.register_model(model_name='best_model', 
            model_path='outputs/model.joblib')

Best run metrics : {'Regularization Strength:': 0.01, 'Max iterations:': 200, 'Accuracy': 0.9121396054628225}
Best run details : {'runId': 'HD_4b351f6b-88c0-46ea-abf3-3a48ed4bf074_8', 'target': 'udacityAzureML', 'status': 'Completed', 'startTimeUtc': '2021-04-17T11:41:34.499205Z', 'endTimeUtc': '2021-04-17T11:42:23.823768Z', 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': 'ca7ce0c5-eaa5-407a-8367-18a958fd6a74', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '0.01', '--max_iter', '200'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'udacityAzureML', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'jobName': None, 'maxRunDurationSeconds': None, 'nodeCount': 1, 'priority': None, 'credentialPassthrough':

In [15]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
data_uri = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=data_uri)

In [16]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [21]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_config = AutoMLConfig(
    compute_target = compute_target,
    experiment_timeout_minutes=15,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    enable_onnx_compatible_models=True,
    n_cross_validations=2)

In [22]:
# Submit your automl run

automl_run = exp.submit(automl_config, show_output = False)
automl_run.wait_for_completion()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
p1,AutoML_320fd1f5-f07a-417d-82df-27d00606516e,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


{'runId': 'AutoML_320fd1f5-f07a-417d-82df-27d00606516e',
 'target': 'udacityAzureML',
 'status': 'Completed',
 'startTimeUtc': '2021-04-17T12:04:01.567407Z',
 'endTimeUtc': '2021-04-17T12:28:26.236935Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'udacityAzureML',
  'AMLSettingsJsonString': '{"path":null,"name":"p1","subscription_id":"f9d5a085-54dc-4215-9ba6-dad5d86e60a0","resource_group":"aml-quickstarts-142925","workspace_name":"quick-starts-ws-142925","region":"southcentralus","compute_target":"udacityAzureML","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"test_size":0.0,"validation_size":0.0,"

In [31]:
# Retrieve and save your best automl model.

automl_best_run, automl_best_model = automl_run.get_output()

print("Best run metrics :", automl_best_run)
# print("Best run details :",automl_run.get_details())
# print("Best run file names :",best_run.get_file_names())

best_automl_model = automl_run.register_model(model_name='best_automl_model')

Best run metrics : Run(Experiment: p1,
Id: AutoML_320fd1f5-f07a-417d-82df-27d00606516e_11,
Type: azureml.scriptrun,
Status: Completed)


In [26]:
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/udacityazureml/code/Users/odl_user_142925/Optimizing_ML_Pipeline_Azure


In [32]:
# Delete cluster
compute_target.delete()